# Field Survey Campaign Progress

In [ ]:
# Parameters cell. Set defaults here
day_obs_min = 20250421
day_obs_max = 20250601

In [ ]:
import numpy as np
import pandas as pd
from astropy.time import Time
import matplotlib.pyplot as plt

In [ ]:
import os
os.environ["no_proxy"] += ",.consdb"

from lsst.summit.utils import ConsDbClient

client = ConsDbClient("http://consdb-pq.consdb:8080/consdb")

In [ ]:
instrument = 'lsstcam'

In [ ]:
bands = {'u': '#0c71ff', 'g': '#49be61', 'r': '#c61c00', 'i': '#ffc200', 'z': '#f341a2', 'y': '#5d0000'}

## All Visits

In [ ]:
visits_query = f'''
    SELECT 
        * 
    FROM 
        cdb_{instrument}.visit1 
    WHERE 
        day_obs >= {day_obs_min} AND day_obs <= {day_obs_max}
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
visits['band'].value_counts()

In [ ]:
visits['img_type'].value_counts()

## Science Program Visits

In [ ]:
visits_query = f'''
    SELECT 
        * 
    FROM 
        cdb_{instrument}.visit1 
    WHERE 
        science_program in ('BLOCK-365')
        AND day_obs >= {day_obs_min} AND day_obs <= {day_obs_max}
'''

visits = client.query(visits_query).to_pandas()

In [ ]:
len(visits)

In [ ]:
visits['target_name'].value_counts()

### Distribution by day_obs

In [ ]:
visits['day_obs'].value_counts().sort_index()

In [ ]:
def dayObsToMJD(day_obs):
    year = np.floor(day_obs / 10000).astype(int)
    month = np.floor((day_obs % 10000) / 100).astype(int)
    day = day_obs % 100
    time = Time({'year': year, 'month': month, 'day': day}, format='ymdhms')
    return time.mjd

In [ ]:
def MJDToDayObs(mjd):
    time = Time(mjd, format='mjd')
    return np.array([_.iso.split()[0].replace('-', '') for _ in time])

In [ ]:
mjd = dayObsToMJD(visits['day_obs'])
xticks = np.arange(np.min(mjd), np.max(mjd) + 2)[::2]
xtick_labels = MJDToDayObs(xticks)

plt.figure()
plt.scatter(dayObsToMJD(visits.value_counts('day_obs').index.values), 
            visits.value_counts('day_obs').values, c='black', label='Total')
for band in bands:
    selection = (visits['band'] == band)
    plt.scatter(dayObsToMJD(visits[selection].value_counts('day_obs').index.values), 
                visits[selection].value_counts('day_obs').values, 
                c=bands[band], label=band)

plt.legend(loc='upper left')
plt.xticks(xticks, xtick_labels, rotation=45.)
plt.ylabel('Visit Count')
plt.grid()
plt.tight_layout()

### Distribution by target and band

In [ ]:
targets = np.unique(visits['target_name'])

In [ ]:
data_target_counts = {
    'target': [],
    'u': [],
    'g': [],
    'r': [],
    'i': [],
    'z': [],
    'y': [],
    'total': []
}

for target in targets:
    data_target_counts['target'].append(target)
    selection_target = (visits['target_name'] == target)
    data_target_counts['total'].append(np.sum(selection_target))
    for band in bands:
        selection_band = (visits['band'] == band)
        data_target_counts[band].append(np.sum(selection_band & selection_target))

df_target_counts = pd.DataFrame(data_target_counts)
df_target_counts

### Distribution by target and band on most recent day

In [ ]:
most_recent_day_obs = np.unique(visits['day_obs'])[-1]
selection_day_obs = (visits['day_obs'] == most_recent_day_obs)
print(f'Observations on day_obs = {most_recent_day_obs}')

In [ ]:
data_target_counts = {
    'target': [],
    'u': [],
    'g': [],
    'r': [],
    'i': [],
    'z': [],
    'y': [],
    'total': []
}

for target in targets:
    data_target_counts['target'].append(target)
    selection_target = (visits['target_name'] == target)
    data_target_counts['total'].append(np.sum(selection_target))
    for band in bands:
        selection_band = (visits['band'] == band)
        data_target_counts[band].append(np.sum(selection_band & selection_target))

df_target_counts = pd.DataFrame(data_target_counts)
df_target_counts

### Distribution by Airmass

In [ ]:
plt.figure()
for target in targets:
    selection_target = (visits['target_name'] == target)
    plt.hist(visits['airmass'][selection_target].values, bins=np.linspace(1., 2., 41), label=target)
plt.legend()
plt.xlabel('Airmass')
plt.ylabel('Counts')

### Distribution by Target, Time, and Airmass

In [ ]:
for target in targets:
    selection_target = (visits['target_name'] == target)
    
    plt.figure()
    for band in bands:
        selection_band = (visits['band'] == band) 
        if np.any(selection_target & selection_band):
            plt.scatter(dayObsToMJD(visits['day_obs'][selection_target & selection_band]), 
                        visits['airmass'][selection_target & selection_band], 
                        c=bands[band], marker='_', label=band)
    plt.legend(loc='lower left')
    plt.xlim(np.min(xticks) - 0.5, np.max(xticks) + 0.5)
    plt.xticks(xticks, xtick_labels, rotation=45.)
    plt.ylabel('Airmass')
    plt.ylim(1.5, 1.0)
    plt.grid()
    plt.title(target)
    plt.tight_layout()